In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
processed_data_path = "../processed_data/"

In [2]:
X_train = pd.read_csv(processed_data_path+"X_train.csv")
X_test = pd.read_csv(processed_data_path+"X_test.csv")
y_train = pd.read_csv(processed_data_path+"y_train.csv")
y_test = pd.read_csv(processed_data_path+"y_test.csv")
y_test = y_test.values.squeeze()

In [3]:
#print(X_train.info())

In [4]:
def score(y, pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for i in range(0, len(y)):
        if int(pred[i]) == 1:
            if int(y[i]) == 1:
                TP = TP + 1
            else:
                FP = FP + 1
        else:
            if int(y[i]) == 1:
                FN = FN + 1
            else:
                TN = TN + 1
    return TP, TN, FP, FN

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=12)

In [6]:
train_data = X_train
train_data['y'] = y_train

#分离正负样本
pos_train_data = train_data[train_data['y'] == 1]
neg_train_data = train_data[train_data['y'] == 0]
pos_train_data.reset_index(drop = True, inplace = True)
neg_train_data.reset_index(drop = True, inplace = True)
#new_count = n * pos_train_data['y'].count()

In [7]:
#print(pos_train_data.reset_index().head())
#print(pos_train_data.reset_index().describe())

In [8]:
from sklearn.neighbors import NearestNeighbors
import random

def smote_up_sample(data,N):    
    #近邻数k
    k = 3
    #上采样倍率N
    N = 1
    #选择用于评估样本点距离的特征
    feature = ['age', 'duration', 'campaign', 'pdays', 'previous','emp.var.rate', 'cons.price.idx', 'cons.conf.idx','euribor3m', 'nr.employed']
          
    #feature = data.columns
    #feature = ['duration','euribor3m','age','nr.employed','campaign','emp.var.rate', 'pdays','education', 'cons.price.idx ']

    #new_data用于存放插值得到的样本
    new_data = np.zeros((data.shape[0]*N,len(feature)))
    newdf = data.copy()
    #print(data.head())
    neighbors = NearestNeighbors(n_neighbors=k).fit(data.loc[:,feature].values)
    for i in range(data.shape[0]):
        #print(data.shape[0]*N)
        point = data.loc[i,feature]
        #print(point.values.reshape(1, -1))
        distances, indices = neighbors.kneighbors(point.values.reshape(1, -1))
        nn = random.randint(1,k-1)
        dis = data.loc[indices[0,nn],feature].values.reshape(1, -1) - data.loc[i,feature].values.reshape(1, -1)
        gap = random.random()
        new_data[i] = data.loc[i,feature].values.reshape(1, -1) + dis * gap
        newdf.loc[i,feature] = new_data[i]             
    data = data.append(newdf)
    return data

In [9]:
#对正样本进行somte过采样
#print(pos_train_data.head())
pos_train_data = smote_up_sample(pos_train_data,N=2)

#pos_train_data = smote_up_sample(pos_train_data)
#简单复制正样本
#pos_train_data = pd.concat([pos_train_data, pos_train_data],sort=False)
#对负样本欠采样
neg_train_data = neg_train_data.sample(frac = 0.5,random_state=42)
#重新拼接正负样本
train_data_sample = pd.concat([pos_train_data, neg_train_data],sort=False)

X_train = train_data_sample.drop(['y'],axis =1)
y_train = train_data_sample['y']


In [10]:
"""
#用PCA做降维处理
from sklearn.decomposition import PCA
pca = PCA(n_components=45)
pca.fit_transform(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
X_val = pca.transform(X_val)
print(X_train.shape)
"""

'\n#用PCA做降维处理\nfrom sklearn.decomposition import PCA\npca = PCA(n_components=45)\npca.fit_transform(X_train)\nX_train = pca.transform(X_train)\nX_test = pca.transform(X_test)\nX_val = pca.transform(X_val)\nprint(X_train.shape)\n'

In [11]:
"""
from sklearn.tree import DecisionTreeClassifier
dc = DecisionTreeClassifier()
dc.fit(X_train, y_train) 
y_train_pred = dc.predict(X_train)
y_test_pred = dc.predict(X_test)
y_val_pred = dc.predict(X_val)
"""

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=600,criterion = "entropy", oob_score=True)
rf.fit(X_train, y_train) 
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)
y_val_pred = rf.predict(X_val)


"""
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=400, learning_rate=1.0,max_depth=1, random_state=0)
gbc.fit(X_train, y_train)
y_train_pred = gbc.predict(X_train)
y_test_pred = gbc.predict(X_test)
y_val_pred = gbc.predict(X_val)
"""

"""
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train) 
y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)
y_val_pred = xgb.predict(X_val)
"""

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


'\nfrom xgboost import XGBClassifier\nxgb = XGBClassifier()\nxgb.fit(X_train, y_train) \ny_train_pred = xgb.predict(X_train)\ny_test_pred = xgb.predict(X_test)\ny_val_pred = xgb.predict(X_val)\n'

In [12]:
import matplotlib.pyplot as plt
#print(X_train.columns,rf.feature_importances_)
f, ax = plt.subplots(figsize=(10, 5))
plt.xticks(rotation=90)
ax.bar(X_train.columns,rf.feature_importances_)
ax.set_title("Feature Importances")
f.show()

D:\ProgramData\Anaconda3\lib\site-packages\matplotlib\figure.py:418: UserWarning: matplotlib is currently using a non-GUI backend, so cannot show the figure
  "matplotlib is currently using a non-GUI backend, "


In [13]:
from sklearn.metrics import classification_report

print("Train set:", classification_report(y_train, y_train_pred))
print("Valiation set:", classification_report(y_val, y_val_pred))
print("Test set:", classification_report(y_test, y_test_pred))

from sklearn.metrics import f1_score
f1_score(y_test, y_test_pred)
TP, TN, FP, FN = score(y_test, y_test_pred)
P = TP / (TP + FP)
R = TP / (TP + FN)
F1 = (2 * P * R) / (P + R)

print("Percision：%f"%P)
print("Recall：%f"%R)
print("F1-score：%f"%F1)

Train set:              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00     10956
        1.0       1.00      1.00      1.00      5602

avg / total       1.00      1.00      1.00     16558

Valiation set:              precision    recall  f1-score   support

        0.0       0.97      0.92      0.94      7296
        1.0       0.55      0.80      0.65       942

avg / total       0.92      0.90      0.91      8238

Test set:              precision    recall  f1-score   support

        0.0       0.97      0.92      0.94      7341
        1.0       0.54      0.73      0.62       897

avg / total       0.92      0.90      0.91      8238

Percision：0.537082
Recall：0.734671
F1-score：0.620527


In [14]:
#F_I = pd.DataFrame(rf.feature_importances_.reshape((1,-1)), columns = X_train.columns)
F_I = pd.DataFrame(rf.feature_importances_.reshape((-1,1)), index = X_train.columns, columns = ['feature_importances'])
print(F_I.columns.tolist())
print(F_I.info())
print(rf.feature_importances_)
print(F_I.sort_values(by = 'feature_importances', axis = 0,ascending = False))

['feature_importances']
<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, age to day_of_week_wed
Data columns (total 1 columns):
feature_importances    50 non-null float64
dtypes: float64(1)
memory usage: 3.3+ KB
None
[6.58988575e-02 2.98112460e-02 5.29338702e-07 1.39759386e-02
 9.90355584e-03 3.41928536e-01 4.05287550e-02 1.99306090e-02
 1.02506931e-02 4.58624441e-02 2.84140974e-02 3.11403093e-02
 1.02600714e-01 5.49982662e-02 5.14652300e-03 6.02284074e-03
 1.45761450e-02 9.55790378e-03 8.49089225e-03 2.98737401e-03
 2.55376215e-03 5.62499358e-03 4.40187486e-03 3.80258684e-03
 5.41842372e-03 3.94564791e-03 8.18617969e-03 2.44272671e-03
 1.23340748e-03 5.54850911e-03 9.65104208e-03 8.76951784e-03
 4.20107581e-04 8.65373727e-03 8.98220171e-03 3.99644146e-03
 2.17362321e-03 5.28607997e-04 2.43965917e-03 2.89268649e-03
 3.79992890e-03 8.88346564e-03 2.37143492e-03 5.62303680e-03
 1.21037019e-03 8.99945307e-03 9.21942491e-03 9.15439180e-03
 8.21727867e-03 8.82924738e-03]
           